# Setup

In [1]:
import torch
from models import DummyGPTModel, GPTConfig
import tiktoken

# Dummy implementation

In [2]:
tokenizer = tiktoken.get_encoding("gpt2")
batch: list[torch.Tensor] = []
txt1 = "Every effort moves you"
txt2 = "Every day holds a"

batch.append(torch.tensor(tokenizer.encode(txt1)))
batch.append(torch.tensor(tokenizer.encode(txt2)))

batch = torch.stack(batch, dim=0)
batch

tensor([[6109, 3626, 6100,  345],
        [6109, 1110, 6622,  257]])